In [1]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(1_000_000_000_000) ## 1TB?

In [2]:
import shared_utils
import pandas as pd
from siuba import *

/opt/conda/lib/python3.10/site-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.


In [3]:
shared_utils.gtfs_utils.get_trips?

Signature:
shared_utils.gtfs_utils.get_trips(
    selected_date: str | datetime.date = datetime.date(2022, 10, 16),
    itp_id_list: list[int] = None,
    trip_cols: list[str] = None,
    get_df: bool = True,
    custom_filtering: dict = None,
) -> pandas.core.frame.DataFrame | siuba.sql.verbs.LazyTbl
Docstring: <no docstring>
File:      ~/data-analyses/_shared_utils/shared_utils/gtfs_utils.py
Type:      function


In [4]:
bbb_trips = shared_utils.gtfs_utils.get_trips(itp_id_list=[300])

In [5]:
bbb_trips >> head(3)

,feed_key,trip_key,trip_id,route_id,calitp_itp_id,calitp_url_number,service_id,service_date,service_indicator,service_start_date,...,direction_id,calitp_deleted_at,block_id,calitp_extracted_at,trip_headsign,trip_short_name,calitp_hash,wheelchair_accessible,shape_id,bikes_allowed
0,1103410659291522770,6029434005241060453,885733,3506,300,0,2,2022-10-16,1,2022-08-14,...,0,2099-01-01,None,2022-09-01,Downtown SM,None,uoygr334lanry9BMZmBV1w==,0,25981,0
1,1103410659291522770,7866236282603963,887182,3515,300,0,2,2022-10-16,1,2022-08-14,...,1,2099-01-01,None,2022-09-01,Playa Vista,None,UNwZQTvljY5jFGURlVb6lQ==,0,26016,0
2,1103410659291522770,-434350452890351870,887657,3519,300,0,2,2022-10-16,1,2022-08-14,...,0,2099-01-01,None,2022-09-01,UCLA,None,u5GBUgMT6howUm0rikuXXQ==,0,26026,0


In [6]:
bbb_routes = shared_utils.gtfs_utils.get_route_info(itp_id_list=[300])

In [7]:
bbb_routes >> head(3)

,feed_key,route_key,date,agency_timezone,agency_id,calitp_deleted_at,agency_url,agency_fare_url,calitp_extracted_at,route_long_name,...,route_type,route_url,calitp_itp_id,route_text_color,route_continuous_drop_off,route_sort_order,calitp_url_number,route_id,route_continuous_pickup,agency_phone
0,1103410659291522770,-6395929695600786447,2022-10-16,America/Los_Angeles,6216179,2099-01-01,http://www.bigbluebus.com,None,2022-07-18,Pacific Palisades,...,3,http://bigbluebus.com/Routes-and-Schedules/Rou...,300,000000,None,None,0,3512,None,310-451-5444
1,1103410659291522770,2548226874833954474,2022-10-16,America/Los_Angeles,6216179,2099-01-01,http://www.bigbluebus.com,None,2022-07-18,Pico Blvd Rapid,...,3,http://bigbluebus.com/Routes-and-Schedules/Rap...,300,000000,None,None,0,3521,None,310-451-5444
2,1103410659291522770,-8514055961334903631,2022-10-16,America/Los_Angeles,6216179,2099-01-01,http://www.bigbluebus.com,None,2022-07-18,Pico Blvd,...,3,http://bigbluebus.com/Routes-and-Schedules/Rou...,300,000000,None,None,0,3510,None,310-451-5444


In [8]:
with_names = bbb_trips >> inner_join(_, bbb_routes >> select(_.route_id, _.route_short_name), on = 'route_id')

In [9]:
daily_trips = with_names >> count(_.route_short_name)
daily_trips

,route_short_name,n
0,1,116
1,14,57
2,17,50
3,18,42
4,2,68
5,3,129
6,7,125
7,8,66
8,9,38
9,R12,79


In [10]:
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual

In [46]:
route_selector = widgets.Dropdown(options = daily_trips.route_short_name,
                description = "Select Route", style= {'description_width': 'initial'})

In [47]:
increase_box = widgets.IntText(value=0, description='Additional Daily Trips', style= {'description_width': 'initial'})

In [91]:
def show_trips(route, additional_trips):
    current_trips = (daily_trips >> filter(_.route_short_name == route)).n.iloc[0]
    new_trips = current_trips + additional_trips
    # these are made up numbers
    base_factor = 50
    rider_factor = base_factor + additional_trips / 5
    
    current_ridership_est = current_trips*base_factor
    new_ridership_est = new_trips*rider_factor
    display(f'Current daily trips: {current_trips}')
    display(f'Trips after increase: {new_trips}, {new_ridership_est - current_ridership_est} new riders')

In [95]:
interact(show_trips, route=route_selector, additional_trips=increase_box);

interactive(children=(Dropdown(description='Select Route', index=9, options=('1', '14', '17', '18', '2', '3', …